In [ ]:
# Importing Libraries
import nltk
nltk.download('stopwords')
nltk.download('punkt')
nltk.download('wordnet')
import re
import string
import numpy as np
import pandas as pd
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.layers import Dense, Dropout, MaxPooling1D, Concatenate, Input, Flatten, Conv1D
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.utils import to_categorical
from sklearn.decomposition import LatentDirichletAllocation as LDA
from sklearn.decomposition import NMF
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics.pairwise import cosine_distances
from sklearn.model_selection import train_test_split
from sklearn import feature_extraction
from gensim import models,corpora
from nltk.corpus import stopwords

In [ ]:
!pip install transformers==3.0.0
!pip install simpletransformers==0.48.0.

In [ ]:
from simpletransformers.classification import ClassificationModel, ClassificationArgs

In [ ]:
%%writefile setup.sh

export CUDA_HOME=/usr/local/cuda-10.1
git clone https://github.com/NVIDIA/apex
pip install -v --no-cache-dir --global-option="--cpp_ext" --global-option="--cuda_ext" ./apex

In [ ]:
!sh setup.sh

In [ ]:
import pandas as pd
import numpy as np

In [ ]:
data = pd.read_csv("../input/covidgurshaan/data_train.csv", encoding = "utf-8-sig")
data_test = pd.read_csv("../input/covidgurshaan/data_test.csv", encoding = "utf-8-sig")

In [ ]:
data = data.drop(columns=['Unnamed: 0'])
data_test = data_test.drop(columns=['Unnamed: 0'])

In [ ]:
data_test.head()

In [ ]:
data_train, data_val = train_test_split(data, test_size=0.15, random_state=42)

In [ ]:
print(len(data_train))
print(len(data_test))
print(len(data_val))

In [ ]:
train_args ={
    'output_dir': 'output/',
    'learning_rate':1e-5,
    'num_train_epochs': 5,
    'reprocess_input_data': True,
    'process_count': 10,
    'train_batch_size': 10,
    'eval_batch_size': 10,
    'max_seq_length': 512,
    'fp16': True,
    'save_steps': 400000,
    'evaluate_during_training': True,
    'evaluate_during_training_verbose': True,
    'save_model_every_epoch': False,
    'save_eval_checkpoints': False,
    'overwrite_output_dir': True
}
model = ClassificationModel('longformer', 'allenai/longformer-base-4096', num_labels=2,  args=train_args)

In [ ]:
model.train_model(data_train, eval_df=data_val)

In [ ]:
X_test = data_test["text"].values
Y_test = data_test["Class"].values
#X_test = pd.DataFrame({"text":X_test[:,0]})

In [ ]:
print(type(Y_test))
print(Y_test)

In [ ]:
predictions,raw = model.predict(X_test.tolist())

In [ ]:
predictions

In [ ]:
from sklearn.metrics import classification_report
print(classification_report(Y_test, predictions))